# Configuration file parser

PhysiCOOL lets you parse the values from the PhysiCell XML configuration file into Python objects which can be modified and used to write new values to the file. Data validation is performed when new values are written to these objects in order to ensure that the new values are consistent with PhysiCell.

Although these classes were developed to be part of the PhysiCOOL black-box model, they can be used on their own to programatically change the values in the PhysiCell configuration file.

## Reading data from the XML file

Reading data from the config file can be done by creating a `ConfigFileParser` instance that takes as input the path to the file. This class has methods to read the different sections of the XML file and store the data in Python objects.

In [6]:
from physicool.config import ConfigFileParser

file_path = "../tests/data/settings.xml"

# Parse the data from the config file
xml_data = ConfigFileParser(file_path)

The data stored in these new objects follows the same structure as the configuration file. Here are some examples:

In [7]:
# Read and print out <domain> data
domain_data = xml_data.read_domain_params()
print(f"Domain data (x-axis): ({domain_data.x_min},{domain_data.x_max})")

# Read and print out <microenvironment_setup> data
substrates = xml_data.read_me_params()
print(f"Decay rate for '{substrates[0].name}': {substrates[0].decay_rate}")

# Read and print out <cell_definitions> data
cell_data = xml_data.read_cell_data(name="default")
print(f"Cell cycle phase durations for 'default': {cell_data.cycle.phase_durations}")
print(f"Cell speed for 'default': {cell_data.motility.speed}")

# Read and print out <user_parameters> data
user_data = xml_data.read_user_params()
for user_param in user_data:
    print(f"{user_param.name}: {user_param.value}")

Domain data (x-axis): (-500,500)
Decay rate for 'substrate': 10.0
Cell cycle phase durations for 'default': [300.0, 480.0, 240.0, 60.0]
Cell speed for 'default': 20.0
random_seed: 0.0
number_of_cells: 5.0


## Writing new data values

Each parameter can also be accessed independently and their values can be changed. The `ConfigFileParser` class includes includes methods to write these changes to the XML file. For instance, the cell speed and the 2D status can be changed with the following commands:

In [8]:
# Update and write the new parameters to the config file
cell_data.motility.speed = 20.0
cell_data.motility.use_2d = True

xml_data.write_motility_params(name="default", motility=cell_data.motility)

When new values are defined, data validation is performed. For instance, if we update the migration bias (which should be between 0 and 1), we get an error stating that the upper bound for this parameter is 1.

In [9]:
cell_data.motility.migration_bias = 2.0

ValidationError: 1 validation error for Motility
migration_bias
  ensure this value is less than or equal to 1.0 (type=value_error.number.not_le; limit_value=1.0)

If several cell parameters are changed, it is possible to use the `write_cell_params` method to update all fields.

In [10]:
cell_data.mechanics.cell_cell_adhesion_strength = 4.0
cell_data.volume.calcification_rate = 1.0

xml_data.write_cell_params(cell_data)